In [1]:
import json
import pandas as pd
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

/Users/sindhu/.pyenv/versions/3.8.18/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('../data/docs-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [3]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [4]:
documents[0]

{'title': 'Label',
 'text': 'Tags objects with identifying attributes that are meaningful and relevant to users.\n\n<!--more--> \n\nLabels are key/value pairs that are attached to objects such as . They are used to organize and to select subsets of objects.',
 'source_file': 'label.md',
 'id': 'b021df6a'}

In [5]:
for doc in tqdm(documents):
    title = doc['title']
    text = doc['text']
    qt = title + ' ' + text

    doc['title_vector'] = model.encode(title)
    doc['text_vector'] = model.encode(text)
    doc['title_text_vector'] = model.encode(qt)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 229/229 [00:42<00:00,  5.34it/s]


In [6]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "title": {"type": "keyword"},
            "source_file": {"type": "text"},
            "id": {"type": "keyword"},
            "title_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "title_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "k8s-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

/var/folders/x1/0r7y2h8j4c52sbjgqxwc9m5c0000gn/T/ipykernel_57390/976164815.py:39: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'k8s-questions'})

In [7]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 229/229 [00:01<00:00, 128.20it/s]


### Hybrid Search

In [8]:
query = "Give me the command to apply a yaml file in Kubernetes"

In [9]:
v_q = model.encode(query)

In [10]:
knn_query = {
    "field": "text_vector",
    "query_vector": v_q,
    "k": 5,
    "num_candidates": 10000,
    "boost": 0.5
}

In [11]:
keyword_query = {
    "bool": {
        "must": {
            "multi_match": {
                "query": query,
                "fields": ["title", "text"],
                "type": "best_fields",
                "boost": 0.5,
            }
        }
    }
}

In [12]:
response = es_client.search(
    index=index_name,
    query=keyword_query,
    knn=knn_query,
    size=5
)

In [13]:
response["hits"]["hits"]

[{'_index': 'k8s-questions',
  '_id': 'tQiXspkBn0oV2v-R4ILW',
  '_score': 7.4470644,
  '_source': {'title': 'Manifest',
   'text': 'Specification of a Kubernetes API object in [JSON](https://www.json.org/json-en.html)\nor [YAML](https://yaml.org/) format.\n\n<!--more-->\nA manifest specifies the desired state of an object that Kubernetes will maintain when you apply the manifest.\nFor YAML format, each file can contain multiple manifests.',
   'source_file': 'manifest.md',
   'id': '76c3e002',
   'title_vector': [-0.01846812292933464,
    0.0012733947951346636,
    0.07594282180070877,
    -0.01765488088130951,
    0.044561076909303665,
    0.06551001965999603,
    0.045482758432626724,
    -0.03412296995520592,
    -0.09205731749534607,
    -0.004754012916237116,
    0.03663255646824837,
    -0.0640902891755104,
    -0.0033857470843940973,
    -0.018710888922214508,
    0.05859145522117615,
    -0.03137584775686264,
    0.07316926121711731,
    -0.020196732133626938,
    0.02698945067

### Hybrid Search Pipeline

In [14]:
df_ground_truth = pd.read_csv('../data/ground-truth-data.csv')

In [15]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [16]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [17]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [18]:
def elastic_search_hybrid(field, query, vector, title):
    knn_query = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "boost": 0.5
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["title", "text"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            },
            "filter": {
                "term": {
                    "title": title
                }
            }
        }
    }

    es_results = es_client.search(
        index=index_name,
        knn=knn_query,
        query=keyword_query,
        size=5,
        _source=["text", "title", "source_file", "id"]
    )

    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [19]:
def question_hybrid(q):
    question = q['question']
    title = q['title']

    v_q = model.encode(question)

    return elastic_search_hybrid('title_vector', question, v_q, title)

In [20]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [21]:
evaluate(ground_truth, question_hybrid)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 944/944 [00:33<00:00, 28.48it/s]


{'hit_rate': 0.9788135593220338, 'mrr': 0.9780190677966102}

#### Keyword Search:```{'hit_rate': 0.9788135593220338, 'mrr': 0.9780190677966102}```

In [22]:
def text_hybrid(q):
    question = q['question']
    title = q['title']

    v_q = model.encode(question)

    return elastic_search_hybrid('text_vector', question, v_q, title)

In [23]:
evaluate(ground_truth, text_hybrid)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 944/944 [00:29<00:00, 32.40it/s]


{'hit_rate': 0.975635593220339, 'mrr': 0.9772245762711864}

#### Keyword Search: ```{'hit_rate': 0.975635593220339, 'mrr': 0.9772245762711864}```

In [24]:
def question_text_hybrid(q):
    question = q['question']
    title = q['title']
    
    v_q = model.encode(question)

    return elastic_search_hybrid('title_text_vector', question, v_q, title)

In [25]:
evaluate(ground_truth, question_text_hybrid)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 944/944 [00:30<00:00, 31.13it/s]


{'hit_rate': 0.975635593220339, 'mrr': 0.9772245762711864}

#### Keyword Search: ```{'hit_rate': 0.975635593220339, 'mrr': 0.9772245762711864}```

## Reranking

In [26]:
def compute_rrf(rank, k=60):
    """ Our own implementation of the relevance score """
    return 1 / (k + rank)

def elastic_search_hybrid_rrf(field, query, vector, k=60):
    knn_query = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "boost": 0.5
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["title", "text"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            }
        }
    }


    knn_results = es_client.search(
        index=index_name, 
        knn=knn_query, 
        size=10
    )['hits']['hits']
    
    keyword_results = es_client.search(
        index=index_name, 
        query=keyword_query, 
        size=10
    )['hits']['hits']
    
    rrf_scores = {}
    # Calculate RRF using vector search results
    for rank, hit in enumerate(knn_results):
        doc_id = hit['_id']
        rrf_scores[doc_id] = compute_rrf(rank + 1, k)

    # Adding keyword search result scores
    for rank, hit in enumerate(keyword_results):
        doc_id = hit['_id']
        if doc_id in rrf_scores:
            rrf_scores[doc_id] += compute_rrf(rank + 1, k)
        else:
            rrf_scores[doc_id] = compute_rrf(rank + 1, k)

    # Sort RRF scores in descending order
    reranked_docs = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Get top-K documents by the score
    final_results = []
    for doc_id, score in reranked_docs[:5]:
        doc = es_client.get(index=index_name, id=doc_id)
        final_results.append(doc['_source'])

    return final_results

In [27]:
def question_text_hybrid_rrf(q):
    question = q['question']
    title = q['title']

    v_q = model.encode(question)

    return elastic_search_hybrid_rrf('title_text_vector', question, v_q)

In [28]:
evaluate(ground_truth, question_text_hybrid_rrf)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 944/944 [00:47<00:00, 20.00it/s]


{'hit_rate': 0.9533898305084746, 'mrr': 0.8777718926553677}

#### Hybrid Search results: ```{'hit_rate': 0.9533898305084746, 'mrr': 0.8777718926553677}```